# EXAMPLE NOTEBOOK: 10 day rolling mean
### Purpose of this notebook is to demonstrate doing a 10-day rolling mean with the data that has been saved to Zarr

In [1]:
import iris
import copy
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import crd_utils as crd

In [2]:
# zarr = '/data/cssp-china/zarr_daily_1851-1859/'
# zarr.storage.ABSStore(container, prefix='', account_name=None, account_key=None, blob_service_kwargs=None)
import zarr
import getpass
account_key = getpass.getpass()


zarr_read = zarr.storage.ABSStore('cssp-china', prefix='zarr_daily_1851-1859', account_name='awsearth', account_key=account_key, blob_service_kwargs=None)

 ························································································


In [3]:
%%time
ds = xr.open_zarr(zarr_read)
ds

CPU times: user 1.39 s, sys: 58.2 ms, total: 1.45 s
Wall time: 3.79 s


<xarray.Dataset>
Dimensions:                                    (grid_latitude: 219, grid_latitude_1: 218, grid_longitude: 286, grid_longitude_1: 286, pressure: 14, time: 3244)
Coordinates:
    forecast_period                            (time) timedelta64[ns] dask.array<chunksize=(204,), meta=np.ndarray>
    forecast_reference_time                    datetime64[ns] ...
  * grid_latitude                              (grid_latitude) float32 22.88 ... -25.08
  * grid_latitude_1                            (grid_latitude_1) float32 22.77 ... -24.969997
  * grid_longitude                             (grid_longitude) float32 323.48 ... 386.18002
  * grid_longitude_1                           (grid_longitude_1) float32 323.59003 ... 386.29004
    height                                     float64 ...
    height_1                                   float64 ...
  * pressure                                   (pressure) float32 10.0 ... 1000.0
  * time                                       (time) datetime64[ns] 1851-01-01T12:00:00 ... 1859-12-28T12:00:00
Data variables:
    air_pressure_at_sea_level                  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_2m_max                  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_2m_mean                 (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_2m_min                  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_pressure_mean           (time, pressure, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 1, 219, 286), meta=np.ndarray>
    cloud_area_fraction                        (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    geopotential_height                        (time, pressure, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 1, 219, 286), meta=np.ndarray>
    m01s05i217_max                             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    precipitation_flux                         (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    relative_humidity_at_2m_max                (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    relative_humidity_at_2m_mean               (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    relative_humidity_at_2m_min                (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    relative_humidity_at_pressure_mean         (time, pressure, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 1, 219, 286), meta=np.ndarray>
    specific_humidity                          (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_air_pressure                       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_downwelling_longwave_flux_in_air   (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_temperature_max                    (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_temperature_mean                   (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    surface_temperature_min                    (time, grid_latitude, grid_l

In [4]:
# What is the size in GBs?
ds.nbytes / 1e9

72.222473716

### Using dask_distributed, let's compute the rolling mean

In [5]:
import os
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import array as da

In [6]:
cluster = KubeCluster()
# cluster.adapt(minimum=1)
cluster.scale(n=10)
cluster

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.244.0.18:40653
distributed.scheduler - INFO -   dashboard at:                     :8787


In [7]:
client = distributed.Client(cluster)
client

distributed.scheduler - INFO - Receive client connection: Client-240ea6a6-597f-11ea-a9ab-7ab1af4de61a
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.244.0.18:40653 Dashboard: /user/kaedonkers/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Create a rolling window for a Dataset using Xarray

In [8]:
vars_3d = copy.deepcopy([name for name in ds.data_vars if not 'pressure' in name])
vars_3d

['air_temperature_at_2m_max',
 'air_temperature_at_2m_mean',
 'air_temperature_at_2m_min',
 'cloud_area_fraction',
 'geopotential_height',
 'm01s05i217_max',
 'precipitation_flux',
 'relative_humidity_at_2m_max',
 'relative_humidity_at_2m_mean',
 'relative_humidity_at_2m_min',
 'specific_humidity',
 'surface_downwelling_longwave_flux_in_air',
 'surface_downwelling_shortwave_flux_in_air',
 'surface_temperature_max',
 'surface_temperature_mean',
 'surface_temperature_min',
 'x_wind_at_10m_mean',
 'y_wind_at_10m_mean']

In [9]:
vars_3d.pop(4)
vars_3d

['air_temperature_at_2m_max',
 'air_temperature_at_2m_mean',
 'air_temperature_at_2m_min',
 'cloud_area_fraction',
 'm01s05i217_max',
 'precipitation_flux',
 'relative_humidity_at_2m_max',
 'relative_humidity_at_2m_mean',
 'relative_humidity_at_2m_min',
 'specific_humidity',
 'surface_downwelling_longwave_flux_in_air',
 'surface_downwelling_shortwave_flux_in_air',
 'surface_temperature_max',
 'surface_temperature_mean',
 'surface_temperature_min',
 'x_wind_at_10m_mean',
 'y_wind_at_10m_mean']

In [53]:
ds_3d = ds[vars_3d[0:2]]
ds_3d

<xarray.Dataset>
Dimensions:                     (grid_latitude: 219, grid_longitude: 286, time: 3244)
Coordinates:
    forecast_reference_time     datetime64[ns] ...
  * time                        (time) datetime64[ns] 1851-01-01T12:00:00 ... 1859-12-28T12:00:00
    height                      float64 ...
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(204,), meta=np.ndarray>
  * grid_longitude              (grid_longitude) float32 323.48 ... 386.18002
  * grid_latitude               (grid_latitude) float32 22.88 22.66 ... -25.08
    height_1                    float64 ...
Data variables:
    air_temperature_at_2m_max   (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_2m_mean  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>

In [54]:
# What is the size in GBs?
ds_3d.nbytes / 1e9

1.625531516

In [55]:
# ds_roll = ds.rolling(time=10, center=False)
ds_roll = ds_3d.rolling(time=10, center=False)
ds_roll

DatasetRolling [window->10,center->False,dim->time]

In [56]:
%%time
ds_rmean = ds_roll.mean()

CPU times: user 456 ms, sys: 35.9 ms, total: 492 ms
Wall time: 1.07 s


In [14]:
# ds_mean_c = ds_rmean.compute()
# ds_mean_c

## Write to a zarr

In [57]:
prefix = 'zarr_rolling_daily_mean_dask'
zarr_path = f'/data/cssp-china/{prefix}'
zarr_write = zarr.storage.ABSStore('cssp-china', prefix=prefix, account_name='awsearth', account_key=account_key, blob_service_kwargs=None)

In [58]:
print(zarr_path)
os.path.isdir(zarr_path)

/data/cssp-china/zarr_rolling_daily_mean_dask


True

In [59]:
!rm -rdf {zarr_path}

In [60]:
os.path.isdir(f'/data/cssp-china/{prefix}')

False

In [61]:
chunks = dict(ds_rmean.dims, time=200)
chunks

{'grid_latitude': 219, 'grid_longitude': 286, 'time': 200}

In [62]:
%%time
ds_rmean = ds_rmean.chunk(chunks=chunks)
ds_rmean.to_zarr(zarr_write, consolidated=True, mode='w')

Client-Request-ID=8e3476ec-5980-11ea-83a7-7ab1af4de61a Retry policy did not allow for a retry: Server-Timestamp=Thu, 27 Feb 2020 16:45:25 GMT, Server-Request-ID=2fb6ab21-401e-0002-328d-ed1657000000, HTTP status code=404, Exception=The specified blob does not exist. ErrorCode: BlobNotFound<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.RequestId:2fb6ab21-401e-0002-328d-ed1657000000Time:2020-02-27T16:45:26.4762012Z</Message></Error>.
Client-Request-ID=8e687262-5980-11ea-83a7-7ab1af4de61a Retry policy did not allow for a retry: Server-Timestamp=Thu, 27 Feb 2020 16:45:26 GMT, Server-Request-ID=2fb6ac6e-401e-0002-578d-ed1657000000, HTTP status code=404, Exception=The specified blob does not exist. ErrorCode: BlobNotFound<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.RequestId:2fb6ac6e-401e-0002-578d-ed1657000000Time:2020-02-27T16:45:26.8194431Z</Message></Error>

CPU times: user 3.88 s, sys: 350 ms, total: 4.23 s
Wall time: 26 s


In [63]:
ds_z = xr.open_zarr(zarr_write)
ds_z

<xarray.Dataset>
Dimensions:                     (grid_latitude: 219, grid_longitude: 286, time: 3244)
Coordinates:
    forecast_period             (time) timedelta64[ns] dask.array<chunksize=(204,), meta=np.ndarray>
    forecast_reference_time     datetime64[ns] ...
  * grid_latitude               (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude              (grid_longitude) float32 323.48 ... 386.18002
    height                      float64 ...
    height_1                    float64 ...
  * time                        (time) datetime64[ns] 1851-01-01T12:00:00 ... 1859-12-28T12:00:00
Data variables:
    air_temperature_at_2m_max   (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>
    air_temperature_at_2m_mean  (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(200, 219, 286), meta=np.ndarray>